In [1]:
import os
from IPython.display import display,Markdown
from openai import OpenAI
from bs4 import BeautifulSoup
import requests

In [2]:
openai=OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")

In [4]:
message="Hello, ollama! this is my first ever message to you! Hi!"
response=openai.chat.completions.create(model="llama3.2",messages=[{'role':'user','content':message}])
response.choices[0].message.content

"Hello and welcome! I'm thrilled to be chatting with you for the very first time. How's your day going so far? Do you have any questions or topics you'd like to discuss? I'm all ears (or rather, all text) and ready to help in any way I can. Let's get this conversation started!"

In [5]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
AI Curriculum
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
If left unchecked, I will happily drone on about LLMs to anyone that will listen. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190 countries. T

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [18]:
system_prompt = """
You are an assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
AI Curriculum
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
If left unchecked, I will happily drone on about LLMs to anyone that will listen. My friends 

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Oh, wow. I bet you didn't see that one coming. Fine, I'll indulge you... the answer to 2 + 2 is... *dramatic pause* ...4. Congratulations, you've managed to calculate a simple math problem. You should be so proud of yourself. Can I go back to more challenging tasks now?


In [12]:
def messages_for(website):
    return [{'role':'system_prompt','content':system_prompt },
    {'role':'user','content':user_prompt_for(website)}]


In [13]:
messages_for(ed)

[{'role': 'system_prompt',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nAI Curriculum\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positiv

In [14]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

'**Summary of the website**\n\n*Title: Home - Edward Donner*\n\nThis website belongs to Ed Donner, a technologist and AI enthusiast.\n* The website showcases Ed\'s work on artificial intelligence (AI) courses and projects.\n\n### Available courses:\n\n- **AI Curriculum**: A full list of courses offered by Ed with over 400,000 enrolled students worldwide.\n\n### News/Announcements\n\n- **"Submit" section**, which updates the following dates:\n  - January 4, 2026\n  - November 11, 2025: \n    * "AI Builder with n8n – Create Agents and Voice Agents"\n  - September 15, 2025: \n    * "The Unique Energy of an AI Live Event"\n\n*   These updates seem to indicate recent events or new releases on Ed Donner\'s platforms.'

In [16]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://edwarddonner.com")

Here is a summary of the website in markdown:

# About Edward Donner's Website
## Overview

This website is written by Ed Donner, the co-founder and CTO of Nebula.io. The site showcases his passion for AI, code development, and self-improvement.

## Features

- **AI Curriculum**: Offers a comprehensive curriculum on LLMs (Large Language Models) through Udemy courses.
- **Outsmart**: An arena where LLMs compete in a battle of diplomacy and deception.
- **Blog/Posts**: Updates Ed Donner shares on his projects, such as AI Builder with n8n, a recent blog post from January 2026.

## News and Announcements

### Recent Post
January 4, 2026 - **AI Builder with n8n – Create Agents and Voice Agents**
This upcoming project involves creating a builder with n8n for AI agents and voice agents. Details are not provided yet, but it suggests an addition to Ed Donner's projects that involve integrating AI in new applications.

### Past Announcements
- November 11, 2025 - The Unique Energy of an AI Live Event
- September 15, 2025 - AI Engineering MLOps Track – Deploy AI to Production
- May 28, 2025 - An announcement on the order in which his best-selling Udemy courses should be taken.

## Contact and Social Media

- Ed Donner's email address: not publicly provided.
- LinkedIn, Twitter, Facebook links are included for connecting with him directly or following updates about his projects.